In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [3]:
import pandas as pd
from pathlib import Path
import os
from factue.utils.viz import disp
import numpy as np

# extract


In [4]:

root = Path("data/llm_output/normalization/extract")
# Example path template
files = root.rglob("**/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]
# df['init_part_no'] = df.source_file.astype(str).str.split('/',expand=True)[9].str.split('_',expand=True)[1].str.split('.',expand=True)[0].astype(int)
df['lang'] = df.source_file.astype(str).str.split('/',expand=True)[8]
df['split'].value_counts()

split
dev    30
Name: count, dtype: int64

In [5]:
df

,text,reference,split,lang,init_part_no,output_id,prompt_name,prompt_version,job,step,...,model_name,model_mode,think_content,is_valid,status,plain_content,raw,claim_candidate,error,source_file
0,"ONE BILLION vaccinated.. thanks to this great man.. honourable PM Modi.. No one could’ve handled the pandemic better than him.. Can’t even imagine the condition,, had this happened before 2014!!",India administered vaccines to 1 billion people by October 2021,dev,dev-eng,34,GPT_41_MINI/v001/default,default,v001,normalization,extract,...,gpt-4.1-mini,chat,"[None, None, None]","[True, True, True]","[no validation, no validation, no validation]","[ONE BILLION vaccinated thanks to honourable PM Modi., ONE BILLION vaccinated thanks to honourable PM Modi., ONE BILLION vaccinated thanks to honourable PM Modi.]","[ONE BILLION vaccinated thanks to honourable PM Modi., ONE BILLION vaccinated thanks to honourable PM Modi., ONE BILLION vaccinated thanks to honourable PM Modi.]","[ONE BILLION vaccinated thanks to honourable PM Modi., ONE BILLION vaccinated thanks to honourable PM Modi., ONE BILLION vaccinated thanks to honourable PM Modi.]",None,data/llm_output/normalization/extract/GPT_41_MINI/v001/default/dev/dev-eng/dev-eng_part_0001.parquet
1,There is NOTHING WRONG with asking why SWING STATES that are run by DEMOCRATS where TRUMP IS WINNING by massive margins (hundreds of thousands of votes) just STOP COUNTING for no reason at all. \n...,The claim: Democrat-led states where Trump was winning stopped counting votes on election night. This is election meddling.,dev,dev-eng,48,GPT_41_MINI/v001/default,default,v001,normalization,extract,...,gpt-4.1-mini,chat,"[None, None, None]","[True, True, True]","[no validation, no validation, no validation]",[There is nothing wrong with asking why swing states run by Democrats where Trump is winning by massive margins just stop counting for no reason at all. This is domestic election interference and ...,[There is nothing wrong with asking why swing states run by Democrats where Trump is winning by massive margins just stop counting for no reason at all. This is domestic election interference and ...,[There is nothing wrong with asking why swing states run by Democrats where Trump is winning by massive margins just stop counting for no reason at all. This is domestic election interference and ...,None,data/llm_output/normalization/extract/GPT_41_MINI/v001/default/dev/dev-eng/dev-eng_part_0001.parquet
2,"Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips!\n.\n.\n.\nCredit: @goodhealthy247 \n#diethacks #fitnessnutrition #weightlosss #fatlossjourney #nutritional #nutriti...",Three carrots give you energy to walk three miles,dev,dev-eng,13,GPT_41_MINI/v001/default,default,v001,normalization,extract,...,gpt-4.1-mini,chat,"[None, None, None]","[True, True, True]","[no validation, no validation, no validation]","[Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov fo...","[Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov fo...","[Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov fo...",None,data/llm_output/normalization/extract/GPT_41_MINI/v001/default/dev/dev-eng/dev-eng_part_0001.parquet
3,EMF Radiation from 5G has & will have an enormous impact on your immune system 😨\nA 2018 study revealed that blocking exposure to electromagnetic fields (EMF’s) produces significantly reduced symp...,Electromagnetic field radiation from 5G has 'an enormous impact on your immune system',dev,dev-eng,46,GPT_41_MINI/v001/default,default,v001,normalization,extract,...,gpt-4.1-mini,ch

In [6]:
sample  = df[['prompt_version','model_name','text','reference', 'claim_candidate','plain_content', 'think_content']].sort_values(['text',"model_name", 'prompt_version'])
sample

,prompt_version,model_name,text,reference,claim_candidate,plain_content,think_content
4,v001,gpt-4.1-mini,2 trillion dollars to train and equip the Afghan military over the past 20 years. They fell in a week. It was never about real training. It was about military contractors and corporations raking i...,The United States spent “2 trillion dollars to train and equip the Afghan military over the past 20 years. They fell in a week.”,"[2 trillion dollars were spent to train and equip the Afghan military over the past 20 years, which fell in a week., 2 trillion dollars were spent to train and equip the Afghan military over the p...","[2 trillion dollars were spent to train and equip the Afghan military over the past 20 years, which fell in a week., 2 trillion dollars were spent to train and equip the Afghan military over the p...","[None, None, None]"
8,v001,gpt-4.1-mini,"BANANAS ARE RADIOACTIVE • • FROM ONE OF D STRANGEST SCIENCE FACTS,BANANAS CONTAIN POTASSIUM,AND SINCE POTASSIUM DECAYS,THAT MAKE THEM SLIGHTLY RAMOACTIVF """,Bananas are radioactive,"[Bananas are radioactive due to the presence of potassium, which decays., Bananas are radioactive because they contain potassium, which decays., Bananas are radioactive because they contain potass...","[Bananas are radioactive due to the presence of potassium, which decays., Bananas are radioactive because they contain potassium, which decays., Bananas are radioactive because they contain potass...","[None, None, None]"
5,v001,gpt-4.1-mini,Don't forget Blade Nzimande is the same age as Pharrell Williams............\n\nDrink responsibly this festives .🙏\n\nFollow the page Don't forget Blade Nzimande is the same age as Pharrell Willia...,Pharrell Williams and Blade Nzimande are the same age,"[Blade Nzimande is the same age as Pharrell Williams., Blade Nzimande is the same age as Pharrell Williams., Blade Nzimande is the same age as Pharrell Williams.]","[Blade Nzimande is the same age as Pharrell Williams., Blade Nzimande is the same age as Pharrell Williams., Blade Nzimande is the same age as Pharrell Williams.]","[None, None, None]"
3,v001,gpt-4.1-mini,EMF Radiation from 5G has & will have an enormous impact on your immune system 😨\nA 2018 study revealed that blocking exposure to electromagnetic fields (EMF’s) produces significantly reduced symp...,Electromagnetic field radiation from 5G has 'an enormous impact on your immune system',"[EMF radiation from 5G has an enormous impact on the immune system, and a 2018 study revealed that blocking exposure to electromagnetic fields significantly reduces symptoms in 90% of patients wit...","[EMF radiation from 5G has an enormous impact on the immune system, and a 2018 study revealed that blocking exposure to electromagnetic fields significantly reduces symptoms in 90% of patients wit...","[None, None, None]"
2,v001,gpt-4.1-mini,"Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips!\n.\n.\n.\nCredit: @goodhealthy247 \n#diethacks #fitnessnutrition #weightlosss #fatlossjourney #nutritional #nutriti...",Three carrots give you energy to walk three miles,"[Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov fo...","[Follow @Dr.Stefan.Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov for the best fat loss, nutrition, and mindset tips., Follow Dr. Stefan Gospodinov fo...","[None, None, None]"
9,v001,gpt-4.1-mini,I would believe your sadness for Reina if you felt the same sadness for the mothers whose children were killed or abducted by the NPA. And then double killed after being silenced after their page ...,"Facebook third-party fact-checkers took down the Facebook page ""Hands Off Our Children.""","[The post claims that if someone expresses sadness for Reina but not for the mothers whose children were killed or abducted by the NPA, they are

In [7]:
# disp(sample)

In [8]:
cols_to_keep = ['text','reference','split','lang','init_part_no','claim_candidate','original_index']
df['original_index'] = df.index
exploded_df = df[cols_to_keep].explode('claim_candidate')
exploded_df['claim_candidate_order'] = exploded_df.groupby(level=0).cumcount()
exploded_df

,text,reference,split,lang,init_part_no,claim_candidate,original_index,claim_candidate_order
0,"ONE BILLION vaccinated.. thanks to this great man.. honourable PM Modi.. No one could’ve handled the pandemic better than him.. Can’t even imagine the condition,, had this happened before 2014!!",India administered vaccines to 1 billion people by October 2021,dev,dev-eng,34,ONE BILLION vaccinated thanks to honourable PM Modi.,0,0
0,"ONE BILLION vaccinated.. thanks to this great man.. honourable PM Modi.. No one could’ve handled the pandemic better than him.. Can’t even imagine the condition,, had this happened before 2014!!",India administered vaccines to 1 billion people by October 2021,dev,dev-eng,34,ONE BILLION vaccinated thanks to honourable PM Modi.,0,1
0,"ONE BILLION vaccinated.. thanks to this great man.. honourable PM Modi.. No one could’ve handled the pandemic better than him.. Can’t even imagine the condition,, had this happened before 2014!!",India administered vaccines to 1 billion people by October 2021,dev,dev-eng,34,ONE BILLION vaccinated thanks to honourable PM Modi.,0,2
1,There is NOTHING WRONG with asking why SWING STATES that are run by DEMOCRATS where TRUMP IS WINNING by massive margins (hundreds of thousands of votes) just STOP COUNTING for no reason at all. \n...,The claim: Democrat-led states where Trump was winning stopped counting votes on election night. This is election meddling.,dev,dev-eng,48,There is nothing wrong with asking why swing states run by Democrats where Trump is winning by massive margins just stop counting for no reason at all. This is domestic election interference and e...,1,0
1,There is NOTHING WRONG with asking why SWING STATES that are run by DEMOCRATS where TRUMP IS WINNING by massive margins (hundreds of thousands of votes) just STOP COUNTING for no reason at all. \n...,The claim: Democrat-led states where Trump was winning stopped counting votes on election night. This is election meddling.,dev,dev-eng,48,There is nothing wrong with asking why swing states run by Democrats where Trump is winning by massive margins just stop counting for no reason at all. This is domestic election interference and e...,1,1
...,...,...,...,...,...,...,...,...
28,"This is the stark and very sad state of Australia.\nI feel for all the people in NSW and VIC suffering through their COVID regime lockdowns.\nLockdowns that have no basis in science, and have been...",Jail inmate requests sentence extension due to Sydney's lockdown,dev,dev-eng,12,A prisoner at Sydney's Long Bay correctional centre has requested an extension to his jail sentence in order to avoid harsh lockdown measures.,28,1
28,"This is the stark and very sad state of Australia.\nI feel for all the people in NSW and VIC suffering through their COVID regime lockdowns.\nLockdowns that have no basis in science, and have been...",Jail inmate requests sentence extension due to Sydney's lockdown,dev,dev-eng,12,Australian prisoners in certain facilities are exempt from COVID-19 lockdowns and public health orders.,28,2
29,"The medical industry literally gives nothing away for free, except flu shots. Not a life saving EpiPen, not insulin, but flu shots!! They want y'all to be injected so bad they entice you with groc...",Flu shot is free and should be regarded as suspicious,dev,dev-eng,15,"The medical industry gives nothing away for free, except flu shots.",29,0
29,"The medical industry literally gives nothing away for free, except flu shots. Not a life saving EpiPen, not insulin, but flu shots!! They want y'all to be injected so bad they entice you with groc...",Flu shot is free and should be regarded as suspicious,dev,dev-eng,15,"The medical industry gives nothing away for free, except flu shots.",29,1
